In [2]:
!pip install -U torch 
!pip install PyMuPDF
!pip install tqdm 
!pip install sentence-transformers
!pip install accelerate 
!pip install bitsandbytes 
!pip install transformers

^C


In [3]:
import os
import requests

# Get PDF document
pdf_path = "./Baseline_Report.pdf"

In [6]:

import fitz
from tqdm.auto import tqdm

def text_formatter(text: str) -> str:
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip() # note: this might be different for each doc (best to experiment)

    # Other potential text formatting functions can go here
    return cleaned_text

# Open PDF and get lines/pages
# Note: this only focuses on text, rather than images/figures etc
def open_and_read_pdf(pdf_path: str) -> list[dict]:
    doc = fitz.open(pdf_path)  # open a document
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):  # iterate the document pages
        text = page.get_text()  # get plain text encoded as UTF-8
        text = text_formatter(text)
        pages_and_texts.append({"page_number": page_number,  # adjust page numbers since our PDF starts on page 42
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_sentence_count_raw": len(text.split(". ")),
                                "page_token_count": len(text) / 4,  
                                "text": text})
    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]


138it [00:00, 222.08it/s]


[{'page_number': 0,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 0.0,
  'text': ''},
 {'page_number': 1,
  'page_char_count': 301,
  'page_word_count': 43,
  'page_sentence_count_raw': 5,
  'page_token_count': 75.25,
  'text': 'i ICAR - Central Research Institute for Dryland Agriculture Santoshnagar, Hyderabad - 500 059, Telangana, India www.icar-crida.res.in A. Amarender Reddy M. Osman V. K. Singh Baseline Survey of SC-Sub Plan  Villages for  Building Local Capabilities A Problem-Driven Iterative Adaptation (PDIA) Approach'}]

In [7]:
import random

random.sample(pages_and_texts, k=3)

[{'page_number': 46,
  'page_char_count': 2530,
  'page_word_count': 449,
  'page_sentence_count_raw': 15,
  'page_token_count': 632.5,
  'text': '36 Average income of STs was the lowest, followed by OBC, SC and highest among FCs (Figure 6.3).  However, income gap among social groups is less than the gap between marginal and large land-class. It  indicates that, the social group as an indicator of economic status is losing relevance, while landholding  size playing a major role in higher incomes.  Figure 6.3: Sources of income by social group Among all social groups, ST households received the lowest share of income from agriculture (49.8 per  cent of total income), which was followed by SCs (60.4 per cent of total income) as against the 71.4 per  cent for FC households. Also, income from the livestock rearing is higher in the other caste group at 7.1  per cent and is the lowest for scheduled tribe at 2.8 per cent. Although agriculture is main source of income (71 per cent) for only FC

In [8]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,0,0,1,1,0.00,
1,1,301,43,5,75.25,i ICAR - Central Research Institute for Drylan...
2,2,725,98,10,181.25,"ii Citation: Reddy Amarender A., Osman, M and..."
3,3,3143,492,20,785.75,iii FOREWORD The Scheduled Castes (SCs) are o...
4,4,2193,332,16,548.25,iv inputs to achieve the target of improved li...


In [11]:
from spacy.lang.en import English # see https://spacy.io/usage for install instructions

nlp = English()

# Add a sentencizer pipeline, see https://spacy.io/api/sentencizer/ 
nlp.add_pipe("sentencizer")

# Create a document instance as an example
doc = nlp("This is a sentence. This another sentence.")
assert len(list(doc.sents)) == 2

# Access the sentences of the document
list(doc.sents)

[This is a sentence., This another sentence.]

In [12]:
for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item["text"]).sents)
    
    # Make sure all sentences are strings
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]
    
    # Count the sentences 
    item["page_sentence_count_spacy"] = len(item["sentences"])

100%|██████████| 138/138 [00:01<00:00, 85.58it/s] 


In [13]:
# Inspect an example
random.sample(pages_and_texts, k=1)

[{'page_number': 42,
  'page_char_count': 1943,
  'page_word_count': 314,
  'page_sentence_count_raw': 21,
  'page_token_count': 485.75,
  'text': '32 F) Investment capabilities of the households The average investment in agriculture on acre basis, Figure 5.13, is observed to be made highest by  the Scheduled Tribe farmers in the Edagatta village (Rs. 30,000) and the lowest by the FC farmers  in Mallampet village (Rs. 15,000). In total sample only one FC household is there, that too, they are  actively engaged in non-farm activities. This is half of what a Scheduled tribe farmer of Edagatta  village farmer pays. This shows that, slowly, farmers who have capabilities in terms of social contacts,  education are slowly investing out of agriculture and trying to settling down in non-agricultural activities  either they are petty business or engaged as contractors in construction or any other works related to  nearby town. Further, the average agriculture investment is higher in Edagatta vi

In [14]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy
count,138.00,138.00,138.00,138.00,138.00,138.00
mean,68.50,1774.13,293.59,14.47,443.53,12.78
std,39.98,869.20,144.57,11.19,217.30,8.76
min,0.00,0.00,1.00,1.00,0.00,0.00
25%,34.25,1128.50,196.25,6.00,282.12,6.00
50%,68.50,1742.50,281.00,13.00,435.62,12.50
75%,102.75,2473.75,403.50,20.00,618.44,17.75
max,137.00,3910.00,655.00,69.00,977.50,45.00


In [15]:
# Define split size to turn groups of sentences into chunks
num_sentence_chunk_size = 10 

# Create a function that recursively splits a list into desired sizes
def split_list(input_list: list, 
               slice_size: int) -> list[list[str]]:
    """
    Splits the input_list into sublists of size slice_size (or as close as possible).

    For example, a list of 17 sentences would be split into two lists of [[10], [7]]
    """
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

# Loop through pages and texts and split sentences into chunks
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

100%|██████████| 138/138 [00:00<?, ?it/s]


In [16]:
# Sample an example from the group (note: many samples have only 1 chunk as they have <=10 sentences total)
random.sample(pages_and_texts, k=1)

[{'page_number': 130,
  'page_char_count': 983,
  'page_word_count': 180,
  'page_sentence_count_raw': 27,
  'page_token_count': 245.75,
  'text': '120 Mancherial District S.No. Parameters Unit Mancherial 1 Key Characteristics a Geographical area Sq. Km. 4,016.46 2 Administrative Units a Revenue villages Nos. 382 b Revenue mandals Nos. 18 c Revenue  divisions Nos. 2 d Gram panchayats Nos. 311 e Mandal praja parishads Nos. 16 f Zilla parishads Nos. 1 g Municipalities Nos. 7 2 Demographic Profile A Population as per 2011 Census a Total Nos. 8,07,037 i Males Nos. 4,08,272 ii Females Nos. 3,98,765 iii Sex Ratio Nos. 977 iv Rural Nos. 4,40,466 v Urban Nos. 3,66,571 vi  Percentage of rural population  % 54.58 vii  Percentage of urban population  % 45.42 B Households Nos. 2,06,983 i Density of the population Nos. 201 3 Child Population (0-6 Years) a Total Nos. 73,725 i Males Nos. 38,578 ii Females Nos. 35,147 iii Rural Nos. 43,204 iv Urban Nos. 30,521 v Sex Ratio Ratio 911 4 Literates a Total

In [17]:
# Create a DataFrame to get stats
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy,num_chunks
count,138.00,138.00,138.00,138.00,138.00,138.00,138.00
mean,68.50,1774.13,293.59,14.47,443.53,12.78,1.76
std,39.98,869.20,144.57,11.19,217.30,8.76,0.84
min,0.00,0.00,1.00,1.00,0.00,0.00,0.00
25%,34.25,1128.50,196.25,6.00,282.12,6.00,1.00
50%,68.50,1742.50,281.00,13.00,435.62,12.50,2.00
75%,102.75,2473.75,403.50,20.00,618.44,17.75,2.00
max,137.00,3910.00,655.00,69.00,977.50,45.00,5.00


In [18]:
import re

# Split each chunk into its own item
pages_and_chunks = []
for item in tqdm(pages_and_texts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]
        
        # Join the sentences together into a paragraph-like structure, aka a chunk (so they are a single string)
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) # ".A" -> ". A" for any full-stop/capital letter combo 
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        # Get stats about the chunk
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 # 1 token = ~4 characters
        
        pages_and_chunks.append(chunk_dict)

# How many chunks do we have?
len(pages_and_chunks)

100%|██████████| 138/138 [00:00<00:00, 6262.93it/s]


243

In [19]:
# View a random sample
random.sample(pages_and_chunks, k=1)

[{'page_number': 56,
  'sentence_chunk': 'The KCC scheme was launched in year 1998 with the aim of providing short-term formal credit to farmers and was created by NABARD (National Bank for Agriculture and Rural Development). The KCC scheme was introduced to ensure that the credit requirements for farmers in the agriculture, fisheries and animal husbandry sector are met. This was done by helping them avail short-term loans and provide them with a credit limit to purchase equipment and for their other expenses as well. Moreover, with the help of KCC, farmers are exempted from the high interest rates of the regular loans offered by banks as the interest rate for KCC starts as low as 2 per cent and averages at 4 per cent. Under this scheme, farmers can repay their loans depending on the harvesting period of their crop for which the loan was given.some other welfare schemes primarily targeted to weaker sections are old-aged, widowed are Aasara pension schemes. It is a welfare scheme of pen

In [20]:
# Get stats about our chunks
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,243.00,243.00,243.0,243.00
mean,71.89,996.16,155.8,249.04
std,39.20,611.31,96.8,152.83
min,1.00,12.00,2.0,3.00
25%,38.50,463.50,71.5,115.88
50%,74.00,962.00,149.0,240.50
75%,105.00,1386.00,218.5,346.50
max,136.00,2701.00,417.0,675.25


In [21]:
# Show random chunks with under 30 tokens in length
min_token_length = 30
for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk token count: 4.75 | Text: Dev, 29(1), 97-106.
Chunk token count: 20.25 | Text: The new Telangana state: A perspective for inclusive and sustainable development.
Chunk token count: 3.0 | Text: 1,000/ acre)
Chunk token count: 15.75 | Text: Food and Nutrition Security 103 19. Summary and Action plan 109
Chunk token count: 25.0 | Text: 1,908 Total 3,937 C Backward Classes a Student Hostels (Including colleges) i Hostels for boys Nos.8


In [22]:
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

[{'page_number': 1,
  'sentence_chunk': 'i ICAR - Central Research Institute for Dryland Agriculture Santoshnagar, Hyderabad - 500 059, Telangana, India www.icar-crida.res.in A. Amarender Reddy M. Osman V. K. Singh Baseline Survey of SC-Sub Plan Villages for Building Local Capabilities A Problem-Driven Iterative Adaptation (PDIA) Approach',
  'chunk_char_count': 299,
  'chunk_word_count': 41,
  'chunk_token_count': 74.75},
 {'page_number': 2,
  'sentence_chunk': 'ii Citation: Reddy Amarender A., Osman, M and Singh, V. K. 2021. Baseline Survey of SC-Sub Plan Villages for Building Local Capabilities - A Problem-Driven Iterative Adaptation (PDIA) Approach. ICAR-Central Research Institute for Dryland Agriculture (CRIDA), Santoshnagar, Hyderabad, Telangana, India.126 p. ISBN : 978-93-80883-61-8 © ICAR-CRIDA 2021 Number of copies: 100 Editorial assistance: Shailesh Borkar, SRF. Published by The Director ICAR-Central Research Institute for Dryland Agriculture (CRIDA) Santoshnagar, Hyderabad-5

In [23]:
# Requires !pip install sentence-transformers
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                      device="cpu")

c:\Users\netha\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\netha\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [28]:

embedding_model.to("cpu")
# doralora
#qlora
#peft

for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])

100%|██████████| 237/237 [01:18<00:00,  3.01it/s]


In [29]:
# Turn text chunks into a single list
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]

In [30]:

# Embed all texts in batches
text_chunk_embeddings = embedding_model.encode(text_chunks,
                                               batch_size=32, # you can use different batch sizes here for speed/performance, I found 32 works well for this use case
                                               convert_to_tensor=True) # optional to return embeddings as tensor instead of array

text_chunk_embeddings

tensor([[ 0.0006, -0.0278, -0.0228,  ...,  0.0434, -0.0553,  0.0088],
        [ 0.0071, -0.0186, -0.0273,  ...,  0.0496, -0.0541,  0.0128],
        [-0.0279,  0.0069, -0.0149,  ...,  0.0543,  0.0065,  0.0235],
        ...,
        [ 0.0055, -0.0125, -0.0208,  ...,  0.0152, -0.0149,  0.0092],
        [-0.0158, -0.0280,  0.0097,  ..., -0.0148, -0.0715,  0.0108],
        [ 0.0066, -0.0220,  0.0110,  ..., -0.0097, -0.0631, -0.0085]])

In [31]:
# Save embeddings to file
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)


In [6]:
# Import saved file and view
import pandas as pd
text_chunks_and_embedding_df_load = pd.read_csv("./text_chunks_and_embeddings_df.csv")
text_chunks_and_embedding_df_load.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,1,i ICAR - Central Research Institute for Drylan...,299,41,74.75,[ 5.51848207e-04 -2.78179329e-02 -2.28000395e-...
1,2,"ii Citation: Reddy Amarender A., Osman, M and ...",719,92,179.75,[ 7.05219246e-03 -1.85662210e-02 -2.72879433e-...
2,3,iii FOREWORD The Scheduled Castes (SCs) are of...,1222,178,305.50,[-2.78851073e-02 6.91951346e-03 -1.48980543e-...
3,3,Legislation to improve the socioeconomic situa...,1891,285,472.75,[-3.49979922e-02 -1.50996065e-02 4.05335915e-...
4,4,iv inputs to achieve the target of improved li...,1967,281,491.75,[-5.91007899e-03 -1.66771859e-02 -3.16942446e-...


### ## part2

In [7]:
import random

import torch
import numpy as np 
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"

# Import texts and embedding df
text_chunks_and_embedding_df = pd.read_csv("text_chunks_and_embeddings_df.csv")

# Convert embedding column back to np.array (it got converted to string when it got saved to CSV)
text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

# Convert texts and embedding df to list of dicts
pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")

# Convert embeddings to torch tensor and send to device (note: NumPy arrays are float64, torch tensors are float32 by default)
embeddings = torch.tensor(np.array(text_chunks_and_embedding_df["embedding"].tolist()), dtype=torch.float32).to(device)
embeddings.shape

torch.Size([237, 768])

In [8]:
text_chunks_and_embedding_df.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,1,i ICAR - Central Research Institute for Drylan...,299,41,74.75,"[0.000551848207, -0.0278179329, -0.0228000395,..."
1,2,"ii Citation: Reddy Amarender A., Osman, M and ...",719,92,179.75,"[0.00705219246, -0.018566221, -0.0272879433, -..."
2,3,iii FOREWORD The Scheduled Castes (SCs) are of...,1222,178,305.50,"[-0.0278851073, 0.00691951346, -0.0148980543, ..."
3,3,Legislation to improve the socioeconomic situa...,1891,285,472.75,"[-0.0349979922, -0.0150996065, 0.000405335915,..."
4,4,iv inputs to achieve the target of improved li...,1967,281,491.75,"[-0.00591007899, -0.0166771859, -0.0316942446,..."


In [9]:
#embeddings[0]

In [10]:
from sentence_transformers import util, SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", 
                                      device=device) # choose the device to load the model to

c:\Users\netha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\netha\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\netha\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [11]:
# 1. Define the query
# Note: This could be anything. But since we're working with a nutrition textbook, we'll stick with nutrition-based queries.
query = "Water resources and utilization"
print(f"Query: {query}")

# 2. Embed the query to the same numerical space as the text examples 
# Note: It's important to embed your query with the same model you embedded your examples with.
query_embedding = embedding_model.encode(query, convert_to_tensor=True)

# 3. Get similarity scores with the dot product (we'll time this for fun)
from time import perf_counter as timer

start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=embeddings)[0]
end_time = timer()

print(f"Time take to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

# 4. Get the top-k results (we'll keep this to 5)
top_results_dot_product = torch.topk(dot_scores, k=5)
top_results_dot_product

Query: Water resources and utilization
Time take to get scores on 237 embeddings: 0.00159 seconds.


torch.return_types.topk(
values=tensor([0.5936, 0.5533, 0.5462, 0.5420, 0.5402]),
indices=tensor([118, 107, 123, 108, 122]))

In [12]:
larger_embeddings = torch.randn(100*embeddings.shape[0], 768).to(device)
print(f"Embeddings shape: {larger_embeddings.shape}")

# Perform dot product across 168,000 embeddings
start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=larger_embeddings)[0]
end_time = timer()

print(f"Time take to get scores on {len(larger_embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

Embeddings shape: torch.Size([23700, 768])
Time take to get scores on 23700 embeddings: 0.00789 seconds.


In [13]:
# Define helper function to print wrapped text 
import textwrap

def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

In [14]:
print(f"Query: '{query}'\n")
print("Results:")
# Loop through zipped together scores and indicies from torch.topk
for score, idx in zip(top_results_dot_product[0], top_results_dot_product[1]):
    print(f"Score: {score:.4f}")
    # Print relevant sentence chunk (since the scores are in descending order, the most relevant chunk will be first)
    print("Text:")
    print_wrapped(pages_and_chunks[idx]["sentence_chunk"])
    # Print the page number too so we can reference the textbook further (and check the results)
    print(f"Page number: {pages_and_chunks[idx]['page_number']}")
    print("\n")

Query: 'Water resources and utilization'

Results:
Score: 0.5936
Text:
64 Water filled in OFR of Sri Agadi Bondhakka Water filled in OFR of Sri Kaveri
Rajam Figure 10.6: OFR after renovation Table 10.1: Estimate for proposed water
resource development activities under SC-Sub Plan S. No. Object ID Village Caste
of the farmers Type of harvesting structure Initial capacity (m3) Capacity (m3)
Water resources created (m3) Survey No. longitude (Decimal degree) latitude
(Decimal degree) Estimated cost (Rs.)1 Talla Mallaih Nagampet SC PT with surplus
weir 1,150 3,300 2,150 230 79.83 18.95 2,29,264 2 Kamera Durgaiah Nagampet SC PT
with surplus weir 1,150 3,100 1,950 2,37/6 79.83 18.95 2,11,837 3 Borukunta
Ankulu Nagampet SC PT with surplus weir 900 2,500 1,600 237/7 79.82 18.95
1,65,968 4 Sangem Rajakka Nagampet SC PT with surplus weir 1,200 3,500 2,300 212
79.83 18.95 2,16,862 5 Kaveri Ellaiah Nagampet SC PT with surplus weir 800 3,100
2,300 277/1/A 79.82 18.96 2,19,536 6 Durgam Mallaiah Nagam

In [18]:
def retrieve_relevant_resources(query: str,
                                embeddings: torch.tensor,
                                model: SentenceTransformer=embedding_model,
                                n_resources_to_return: int=5,
                                print_time: bool=True):
    """
    Embeds a query with model and returns top k scores and indices from embeddings.
    """

    # Embed the query
    query_embedding = model.encode(query, 
                                   convert_to_tensor=True) 

    # Get dot product scores on embeddings
    start_time = timer()
    dot_scores = util.dot_score(query_embedding, embeddings)[0]
    end_time = timer()

    if print_time:
        print(f"[INFO] Time taken to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

    scores, indices = torch.topk(input=dot_scores, 
                                 k=n_resources_to_return)

    return scores, indices
def print_top_results_and_scores(query: str,
                                 embeddings: torch.tensor,
                                 pages_and_chunks: list[dict]=pages_and_chunks,
                                 n_resources_to_return: int=5):
    """
    Takes a query, retrieves most relevant resources and prints them out in descending order.

    Note: Requires pages_and_chunks to be formatted in a specific way (see above for reference).
    """
    
    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings,
                                                  n_resources_to_return=n_resources_to_return)
    
    print(f"Query: {query}\n")
    print("Results:")
    # Loop through zipped together scores and indicies
    for score, index in zip(scores, indices):
        print(f"Score: {score:.4f}")
        # Print relevant sentence chunk (since the scores are in descending order, the most relevant chunk will be first)
        print_wrapped(pages_and_chunks[index]["sentence_chunk"])
        # Print the page number too so we can reference the textbook further and check the results
        print(f"Page number: {pages_and_chunks[index]['page_number']}")
        print("\n")
def get_top_results(query: str,
                    embeddings: torch.tensor,
                    pages_and_chunks: list[dict]=pages_and_chunks,
                    n_resources_to_return: int=5) -> list[str]:
    """
    Takes a query, retrieves the most relevant resources and stores them in a list.

    Note: Requires pages_and_chunks to be formatted in a specific way (see above for reference).
    
    Returns:
        results_list (list[str]): A list of sentence chunks containing the results.
    """
    
    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings,
                                                  n_resources_to_return=n_resources_to_return)
    
    results_list = []
    
    for index in indices:
        result = pages_and_chunks[index]["sentence_chunk"]
        results_list.append(result)
    
    return results_list


In [19]:
query = ". Credit utilization pattern"

# Get just the scores and indices of top related results
scores, indices = retrieve_relevant_resources(query=query,
                                              embeddings=embeddings)
scores, indices

[INFO] Time taken to get scores on 237 embeddings: 0.00025 seconds.


(tensor([0.3886, 0.3798, 0.3325, 0.3145, 0.3075]),
 tensor([177, 182, 214, 179, 178]))

In [20]:
# Print out the texts of the top scores
print_top_results_and_scores(query=query,
                             embeddings=embeddings)
result=get_top_results(query=query,
                          embeddings=embeddings)

[INFO] Time taken to get scores on 237 embeddings: 0.00030 seconds.
Query: . Credit utilization pattern

Results:
Score: 0.3886
95 Chapter 16 CREDIT UTILIZATION PATTERN A. Amarender Reddy, R. V. Adake and Ch.
Lavanya ICAR-Central Research Institute for Dryland Agriculture (CRIDA),
Santoshnagar, Hyderabad- 500 059, Telangana State Majority of the Indian farmers
are small and marginal and lack of necessary capital to invest in agriculture
(Reddy, 2012; Kumar, 2013; Ramprasad, 2019). Hence, availability of credit at
low interest rates are crucial for adoption of various inputs and also to meet
daily consumption needs. The availability of credit determines input use
pattern, technology adoption, investments in land development, cost of
cultivation and farm incomes (Cole, 2009; Reddy et al.,2020; Reddy et al.,2021).
In general farmers first preference is formal sources like commercial banks,
Regional Rural banks (RRBs), cooperatives, then from the semi-formal
institutions like Self-Help Gro

In [53]:
#!pip install psutil

In [23]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Define the paths to your local model and tokenizer directories
model_path = 'cache1/model/gpt2'
tokenizer_path = 'cache1/tokenizer/gpt2'

# Initialize the tokenizer and model from the local directories
tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
model = GPT2LMHeadModel.from_pretrained(model_path)
model.eval()  # Set the model to evaluation mode

# Function to combine query and relevant texts into a single input string
def prepare_input(query, relevant_texts):
    context = ' '.join(relevant_texts)
    input_text = f"{query}\n\nRelevant Information:\n{context}"
    return input_text

# Function to generate a response
def generate_response(input_text, tokenizer, model):
    inputs = tokenizer.encode(input_text, return_tensors='pt')
    with torch.no_grad():
        outputs = model.generate(inputs, max_length=500, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Example usage
query = ". Credit utilization pattern"
relevant_texts =result

# Prepare input
input_text = prepare_input(query, relevant_texts)

# Generate response
response = generate_response(input_text, tokenizer, model)
print(response)




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
c:\Users\netha\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\utils.py:1363: UserWarning: Input length of input_ids is 985, but `max_length` is set to 500. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


. Credit utilization pattern

Relevant Information:
95 Chapter 16 CREDIT UTILIZATION PATTERN A. Amarender Reddy, R. V. Adake and Ch. Lavanya ICAR-Central Research Institute for Dryland Agriculture (CRIDA), Santoshnagar, Hyderabad- 500 059, Telangana State Majority of the Indian farmers are small and marginal and lack of necessary capital to invest in agriculture (Reddy, 2012; Kumar, 2013; Ramprasad, 2019). Hence, availability of credit at low interest rates are crucial for adoption of various inputs and also to meet daily consumption needs. The availability of credit determines input use pattern, technology adoption, investments in land development, cost of cultivation and farm incomes (Cole, 2009; Reddy et al.,2020; Reddy et al.,2021). In general farmers first preference is formal sources like commercial banks, Regional Rural banks (RRBs), cooperatives, then from the semi-formal institutions like Self-Help Groups (SHGs) and last preference is informal sources like friends and family, 